# QM/MM in OpenMM

One side effect of interfacing EMLE with OpenMM through Sire is that is now possible to perform 'normal' QM/MM using OpenMM (which has not been previously available). This includes the same 'black-box' implementation for hydrogen link-atoms across covalent bonds between the QM and MM region as available in AMBER (sander). 

Note that when using the EMLEcalculator interface, such QM/MM calculations still use EMLE for the electrostatic embedding (instead of including the MM partial charges directly in the QM calculations, as is performed in standard QM/MM). To run 'standard' QM/MM with the XTB code, a full xtb QM engine for use with Sire should be created, which can be done following the approach shown [here](https://sire.openbiosim.org/tutorial/part08/01_intro.html#creating-a-qm-engine).

In this tutorial, we will first go through performing a QM/MM simulation with XTB2 (GFN2-xTB), using EMLE-embedding, and then perform the equivalent simulation with a MACE MLP. This will be done for the same AbyU system as before. However, in these examples, we also will show that the ML region does not need to be a whole molecule. This is important for most enzyme reactions, where amino-acid side chains also take part. 

In this instance, the QM (or ML) region will be the same AbyU substrate as before, but we will now also include the side-chain of a tryptophan residue that is next to the substrate. 

In [1]:
import math
import os
import sys

import numpy as np

import openmm
import openmm.unit as unit
from openmm.app import *
from openmm import CustomBondForce, CustomCVForce

from emle.models import MACEEMLE
from emle.calculator import EMLECalculator

import sire as sr

distances = ((2125, 2094, 0.7), (2119, 2087, 0.3))
k = (125*unit.kilocalorie_per_mole/unit.angstrom**2).value_in_unit(unit.kilojoule_per_mole/unit.nanometer**2)
r0 = 2.5*unit.angstroms

cv = CustomBondForce("weight*r") 
cv.addPerBondParameter("weight") 

for atom1, atom2, weight in distances: 
    cv.addBond(atom1, atom2, [weight]) 

energy_expression = "k*(weighted_distance - r0)^2"
restraint_force = openmm.CustomCVForce(energy_expression)
restraint_force.addCollectiveVariable("weighted_distance", cv)
restraint_force.addGlobalParameter("k", k)
restraint_force.addGlobalParameter("r0", r0)

/home/elliot/mambaforge/envs/emle-train-sire/lib/python3.12/site-packages/e3nn/o3/_wigner.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  _Jd, _W3j_flat, _W3j_indices =

1

# Creating the system

Again, this is similar to what you have seen before. XTB2 is a supported backend for EMLE already (though you can also use ORCA as a backend, or custom MLPs so long as they support ASE; further details can be found in the [EMLE documentation](https://chemle.github.io/emle-engine/)).

As the QM region now incorporates the tryptophan side-chain, the atom selection must change. Whilst before, we could simply select the seperate substrate molecule, now we must list the actual atoms involved (here as ranges of atom indices) as they are across two different molecules (the enzyme and the substrate). 

For the creation of an OpenMM context and the production run, there are no changes from before. 

In [2]:
#Loads AbyU system
mols = sr.load([f"AbyU_OpenMM.rst", f"AbyU_OpenMM.prm7"])

# Load the topology with OpenMM too.
prm = AmberPrmtopFile(f"AbyU_OpenMM.prm7")

#Create calculator
#calculator = EMLECalculator(device="cpu", backend="xtb")
calculator = EMLECalculator(device="cpu", mace_model="models/abyu_mace_link.model", backend="mace")

# Create an EMLEEngine bound to the calculator on the substrate.
mols, engine = sr.qm.emle(mols, "atomnum 1804:1822,2083:2132", calculator, redistribute_charge=True)


/home/elliot/mambaforge/envs/emle-train-sire/lib/python3.12/site-packages/emle/models/_mace.py:212: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  source_model = _torch.load(

In [3]:
# Create a QM/MM dynamics object.
d = mols.dynamics(
    timestep="1fs",
    constraint="none",
    integrator="langevin_middle",
    temperature="300K",
    qm_engine=engine,
    map={"threads": 1},
    fixed="not atoms within 20A of atomidx 3 in mol[1]"
)

In [4]:
# Get the underlying OpenMM context.
context = d._d._omm_mols

# Get the OpenMM system.
omm_system = context.getSystem()

# Store a copy of the integrator.
integrator = context.getIntegrator().__copy__()

# Add the forces to the OpenMM system.
omm_system.addForce(restraint_force)

# Create a new context.
new_context = openmm.Context(omm_system, integrator, context.getPlatform())

# Set force constant K for the biasing potential.
new_context.setParameter("k", k)

#Set center of biasing potential
new_context.setParameter("r0", r0)

# Set the postions of the new context to be the same as the original context.
new_context.setPositions(context.getState(getPositions=True).getPositions())



# Running the simulation
Here we've saved the trajectory file as "AbyU_OpenMM_xtb.dcd". 

For loading the trajectory in NGLviewer, we have also added in the tryptophan side chain (you can see that the atom listing is the same as when we define the "QM" or "ML" region.). 

Once you've run the XTB simulation, we suggest that you edit code to swap calculators to the MACE MLP in the system setup, and then you can rerun this simulation using ML/MM. (In this instance the MLP was trained to the XTB2 level for ease of training, but you should see that they produce very similar results). 

For viewing the resulting trajectory, you should change the file name in the "file_handle" and trajectory loading below.

In [5]:
# Sampling production. Trajectories are saved in dcd files.
file_handle = open(f"./AbyU_OpenMM/AbyU_OpenMM_mace.dcd", "bw")
state = new_context.getState(getPositions=True)
positions = state.getPositions()
dcd_file = DCDFile(file_handle, prm.topology, dt=integrator.getStepSize())
dcd_file.writeModel(positions)

for x in range(300):
    integrator.step(1)
    state = new_context.getState(getPositions=True)
    positions = state.getPositions()
    dcd_file.writeModel(positions)
        
file_handle.close()

/home/elliot/mambaforge/envs/emle-train-sire/lib/python3.12/site-packages/torch/nn/modules/module.py:1562: UserWarning: at::frobenius_norm is deprecated and it is just left for JIT compatibility. It will be removed in a future PyTorch release. Please use `linalg.vector_norm(A, 2., dim, keepdim)` instead (Triggered internally at /home/conda/feedstock_root/build_artifacts/libtorch_1729124132187/work/aten/src/ATen/native/LinearAlgebra.cpp:3072.)
  return forward_call(*args, **kwargs)


In [6]:
traj = sr.load("AbyU_OpenMM.prm7", "AbyU_OpenMM/AbyU_OpenMM_mace.dcd")

#To view the whole system
#traj.trajectory().view()

# To view just the substrate
traj.atoms()["atomnum 1804:1822,2083:2132"].trajectory().view()

NGLWidget(max_frame=300)